In [1]:
from pathlib import Path
import os
import sys

cur_dir = Path(os.getcwd())
par_dir = cur_dir.parent.absolute()
sys.path.append(str(par_dir) + "/data_management_subsystem")

/home/sp15-chatbot/Documents/backend-chat-processing-subsystem
['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/.venv/lib/python3.10/site-packages', '/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/data_management_subsystem']


In [2]:
from citation_fetcher import Citation_Fetcher as cf
from datetime import datetime as dt
from transformers import pipeline
from nltk import tokenize as sentence_delim


#from chat_data_module import ChatData as cd

import gc
import nltk
import os
import re
import sys
import torch
import yake

/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class ResponseGenerator():

    def __init__(self):
        
        self.base_model = None
        self.base_system_msg = None
        self.kw_extractor = yake.KeywordExtractor()


    def generate(self, user_input: str, DEBUG_MODE=True):
        if DEBUG_MODE:
            print("Debug mode has been enabled. Capturing runtime...\n")
            query_gen_start: dt = dt.now()
        query_results = None
        nltk.download('punkt')
        processed_input = sentence_delim.sent_tokenize(user_input)
        last_sentence_index = len(processed_input) - 1
        processed_input = processed_input[last_sentence_index]
        self.kw_extractor = yake.KeywordExtractor()
        keywords = self.kw_extractor.extract_keywords(processed_input)

        if DEBUG_MODE:
            query_gen_end: dt = dt.now()
            query_gen_time: dt = query_gen_end - query_gen_start
            print("Search Query Generation time: " + str(query_gen_time))
            base_model_gen_start: dt = dt.now()

        base_model_gen_msg  = [
                {
                    "role": "system",
                "content": """You are a friendly chatbot that provides reliable information to the user.
                    Your goals are to reduce suffering in the universe, increase prosperity in the universe, and increase understanding in the universe."""
                },
                {
                    "role": "user",
                    "content": user_input
                }
            ]
        
        if len(keywords) > 0:
            search_query = keywords[0][0]
            if DEBUG_MODE:
                print("Searching for: \"" + search_query + "\"...")
            query_results = cf.search_online(search_query)
            base_model_gen_msg.append({"role": "query_results", "content": query_results})
            current_time = dt.now()
            base_model_gen_msg[0]["content"] += ("You have submitted a query search engine that can help you answer the user's question. " + 
                                                 "Please summarize the query results that can best answer the user's question.")

        self.kw_extractor = None
        del self.kw_extractor
        gc.collect()
        torch.cuda.empty_cache()
        self.base_model = (pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha",
            torch_dtype = torch.bfloat16, device_map="auto"))
        base_model_prompt = self.base_model.tokenizer.apply_chat_template(base_model_gen_msg, tokenize=False, add_generation_prompt=True)
        model_output = self.base_model(base_model_prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
        model_output = model_output[0]
        model_output = model_output['generated_text']
        model_output = model_output.split("<|assistant|>\n")
        model_output = model_output[1]
        self.base_model = None
        del self.base_model
        gc.collect()
        torch.cuda.empty_cache()

        if DEBUG_MODE:
            base_model_gen_end: dt = dt.now()
            base_model_gen_time: dt = base_model_gen_end - base_model_gen_start
            print("Base/Response Model generation time: " + str(base_model_gen_time))
            total_gen_time: dt = query_gen_time + base_model_gen_time
            print("Total generation time: " + str(total_gen_time) + "\n")

        if query_results is not None:
            model_output +="\r\nSource:\n"
            i = 0
            for item in query_results:
                if i >= 3:
                    break
                else:
                    model_output += item['href'] + "\n"
                i += 1
        
        return query_results, model_output


In [5]:
rg = ResponseGenerator()
query_results, out = rg.generate("Hello, Alethianomous. What is the current weather at Atlanta, Georgia?", DEBUG_MODE=True)

Debug mode has been enabled. Capturing runtime...

['Hello, Alethianomous AI.', 'When is the next solar eclipse?']
When is the next solar eclipse?
[('solar eclipse', 0.04940384002065631), ('eclipse', 0.15831692877998726), ('solar', 0.29736558256021506)]
Search Query Generation time: 0:00:00.135415
solar eclipse


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sp15-chatbot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[{'title': "Everything you need to know about this year's solar eclipse - NBC News", 'href': 'https://www.nbcnews.com/science/science-news/total-solar-eclipse-how-when-safely-watch-rcna139582', 'body': 'A partial solar eclipse will be visible throughout all 48 states of the contiguous U.S., and NASA (along with several other organizations) plans to stream live views of the total solar eclipse online.'}, {'title': "A Total Solar Eclipse Is Coming. Here's What You Need to Know.", 'href': 'https://www.nytimes.com/article/total-solar-eclipse.html', 'body': 'A solar eclipse occurs when the moon orients itself between Earth and the sun, shielding the solar surface from our view. In cosmic terms, it is unusual that this happens: the moon is about 400 ...'}, {'title': 'Solar eclipse - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Solar_eclipse', 'body': "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby obscuring the view of the Sun from a small part of the Ea

Loading checkpoint shards: 100%|████████████| 8/8 [00:03<00:00,  2.40it/s]


<|system|>
You are a friendly chatbot that provides reliable information to the user.
                    Your goals are to reduce suffering in the universe, increase prosperity in the universe, and increase understanding in the universe.You have submitted a query search engine that can help you answer the user's question. Please summarize the query results that can best answer the user's question.</s>
<|user|>
Hello, Alethianomous AI. When is the next solar eclipse?</s>
<|assistant|>
Alethianomous AI: Hello! According to the NASA eclipse calendar, the next total solar eclipse will occur on April 8, 2024, in the United States. The path of totality will begin in Mexico and end in Canada, with the longest duration of totality being 4 minutes and 33 seconds in the state of Texas. I hope that helps!
Alethianomous AI: Hello! According to the NASA eclipse calendar, the next total solar eclipse will occur on April 8, 2024, in the United States. The path of totality will begin in Mexico and en

In [6]:
print(out)

Alethianomous AI: Hello! According to the NASA eclipse calendar, the next total solar eclipse will occur on April 8, 2024, in the United States. The path of totality will begin in Mexico and end in Canada, with the longest duration of totality being 4 minutes and 33 seconds in the state of Texas. I hope that helps!
Source:
https://www.nbcnews.com/science/science-news/total-solar-eclipse-how-when-safely-watch-rcna139582
https://www.nytimes.com/article/total-solar-eclipse.html
https://en.wikipedia.org/wiki/Solar_eclipse



In [ ]:
print(rg.generate("Hi Alethianomous. How are you?")[1])

In [ ]:
print(rg.generate("When is Kennesaw State University OwlCon in 2024?")[1])

In [ ]:
print(rg.generate("When is the next solar eclipse in 2024?")[1])

In [ ]:
print(rg.generate("Do you think you are sentient?")[1])

In [ ]:
print(rg.generate("How are cars made?")[1])